<a href="https://colab.research.google.com/github/VladimirVladetic/AgeAndGenderPredictor/blob/main/EDAandModelTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import os
import cv2
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D, Input, BatchNormalization
from PIL import Image
from keras.preprocessing.image import load_img, img_to_array
import zipfile
from keras.utils import to_categorical
from tensorflow.keras.regularizers import l1_l2, l1, l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import save_model, load_model

In [2]:
from google.colab import files

uploaded = files.upload()

source_path = '/content/kaggle.json'
destination_path = '/root/.kaggle/kaggle.json'
kaggle_dir = '/root/.kaggle/'
if not os.path.exists(kaggle_dir):
    os.makedirs(kaggle_dir)
os.rename(source_path, destination_path)

Saving kaggle.json to kaggle.json


In [3]:
! kaggle datasets download -d jangedoo/utkface-new

with zipfile.ZipFile('/content/utkface-new.zip', 'r') as zip_ref:
  zip_ref.extractall('/content')

100% 330M/331M [00:01<00:00, 173MB/s]
100% 331M/331M [00:01<00:00, 186MB/s]


In [4]:
## Function returns paths to images and their associated age and gender labels
def get_image_paths_age_gender_labels(image_directory):
    image_paths = []
    age_labels = []
    gender_labels = []
    for filename in tqdm(os.listdir(image_directory )):
        image_path = os.path.join(image_directory,filename)
        temporary_data = filename.split("_")
        age = int(temporary_data[0])
        gender = int(temporary_data[1])
        image_paths.append(image_path)
        age_labels.append(age)
        gender_labels.append(gender)
    return image_paths, age_labels, gender_labels

In [5]:
def map_age_to_category(age):
  age_ranges = [(0, 2), (3, 9), (10, 20), (21, 29), (30, 45), (46, 60), (61, 80), (81, 120)]
  for category, (min_age, max_age) in enumerate(age_ranges, start=0):
          if min_age <= age <= max_age:
              return category
  return 0

In [6]:
# Extracting features from the images with face detection
def extract_features(images, height, width):
    features = []
    no_face_indices = []  # Keep track of indices for images with no detected faces

    # Load the pre-trained Haarcascades face classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    for idx, image in tqdm(enumerate(images)):
        # Load the image using PIL
        img = load_img(image, grayscale=True)

        # Convert PIL image to a numpy array
        img_array = np.array(img)

        # Detect faces in the image
        faces = face_cascade.detectMultiScale(img_array, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # If faces are detected, extract features from the first detected face
        if len(faces) > 0:
            x, y, w, h = faces[0]
            face_img = img_array[y:y+h, x:x+w]

            # Resize the face image to the specified height and width
            face_img = cv2.resize(face_img, (height, width))

            # Normalize the face image
            face_img = face_img / 255.0

            # Add the face features to the list
            features.append(face_img)
        else:
            # If no faces are detected, add the index to the no_face_indices list
            no_face_indices.append(idx)

    features = np.array(features)
    features = features.reshape(len(features), height, width, 1)
    features = np.array(features, dtype=np.float32)

    return features, no_face_indices


In [7]:
image_directory = "/content/UTKFace/"

image_paths, age_labels, gender_labels = get_image_paths_age_gender_labels(image_directory)

age_categories = [map_age_to_category(age) for age in age_labels]

gender_dictionary = {0:"Male",1:"Female"}

  0%|          | 0/23708 [00:00<?, ?it/s]

In [9]:
## Structure of df: INDEX, PICTURE FILEPATH, AGE LABEL, GENDER LABEL
df = pd.DataFrame()
df["image"], df["age"], df["gender"] = image_paths, age_categories, gender_labels
##print(df)

In [10]:
image_height = 128
image_width = 128
num_age_classes = 8

In [ ]:
result = extract_features(df["image"],image_height,image_width)

X = result[0]
indexlist = result[1]

In [ ]:
df_filtered = df.drop(indexlist)

df_filtered.reset_index(drop=True, inplace=True)

y_gender = np.array(df_filtered["gender"])
y_age = np.array(df_filtered["age"])

y_age = np.array(list(df_filtered["age"].apply(lambda x: to_categorical(x, num_classes=num_age_classes))))

y_gender_tensor = tf.convert_to_tensor(y_gender, dtype=tf.float32)
y_age_tensor = tf.convert_to_tensor(y_age, dtype=tf.float32)

input_shape = (image_height,image_width,1)